In [12]:
def get_deepseek_prompt(english_input: str) -> str:
    return f"""User:The motor has a broken piston. Let's get a repairman to fix it.

    Assistant:個摩打有條活塞爆咗，揾個師傅嚟整整佢。<｜end▁of▁sentence｜>User:He has a husky, magnetic voice.

    Assistant:佢把聲沙沙哋、沉沉哋，好有磁性。<｜end▁of▁sentence｜>User:A sugar cube is about the same amount as a teaspoon.

    Assistant:一粒方糖差唔多係一茶匙糖咁滯。<｜end▁of▁sentence｜>User:{english_input}

    Assistant:"""

In [15]:
def get_llama2_prompt(english_input: str) -> str:
    return f"""<s>[INST] <<SYS>>You are are a helpful assistant that translate English into Cantonese.<</SYS>>
The motor has a broken piston. Let's get a repairman to fix it. [/INST] 個摩打有條活塞爆咗，揾個師傅嚟整整佢。 </s>
<s>[INST] He has a husky, magnetic voice. [/INST] 佢把聲沙沙哋、沉沉哋，好有磁性。 </s>
<s> [INST] A sugar cube is about the same amount as a teaspoon. [/INST] 一粒方糖差唔多係一茶匙糖咁滯。 </s>
<s> [INST] {english_input} [/INST]"""

In [35]:
import subprocess

def infer(english_input: str) -> str:
    # prompt = get_deepseek_prompt(english_input)
    prompt = get_llama2_prompt(english_input)
    result = subprocess.run([
        "../llama.cpp/main",
        "-t", "10",
        "-ngl", "64",
        "-b", "512",
        "--n-predict", "50",
        # "-m", "models/deepseek-llm-7b-chat.Q5_K_M.gguf",
        "-m", "../../../../models/llama-2-7b-chat.Q5_K_M.gguf",
        "--seed", "1",
        "--temp", "0",
        "--repeat_penalty", "1.1",
        "-p", prompt
    ], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout = result.stdout.decode('utf-8', errors='replace')
    # return stdout.split("\n\n    Assistant:")[-1]
    return stdout.split("[/INST] ")[-1]


In [36]:
infer("I am a student.")

'你是學生。'

# Start the Test

In [22]:
from tqdm import tqdm

In [37]:
with open("wordshk-eng-yue/plus15/test.en.txt", "r") as test_file:
    test_lines = test_file.readlines()

with open("results-3-shot.txt", "w") as results_file:
    for line in tqdm(test_lines):
        result = infer(line.strip())
        # Escape newlines just in case to not mess up line counts
        results_file.write(result.replace("\n", "\\n") + "\n")
        results_file.flush()

100%|██████████| 1500/1500 [50:40<00:00,  2.03s/it]
